## Paramters

In [63]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-5678415b1c0fd33e'
PROJECT = 'qwiklabs-gcp-5678415b1c0fd33e'
REGION = 'us-central1'
SEQ_LEN = 240
N_FORWARD = 120

In [64]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['SEQ_LEN'] = str(SEQ_LEN)
os.environ['N_FORWARD'] = str(N_FORWARD)
os.environ['TFVERSION'] = '1.13'

## Train model locally
Make sure the code works as intended

In [57]:
%%bash
OUTDIR=$(pwd)/trained_${SEQ_LEN}_${N_FORWARD}/linear
rm -rf $OUTDIR

gcloud ml-engine local train \
   --module-name=model.task \
   --package-path=${PWD}/model \
   -- \
   --model=linear \
   --train_data_path="data/train_${SEQ_LEN}_${N_FORWARD}.csv" \
   --eval_data_path="data/eval_${SEQ_LEN}_${N_FORWARD}.csv"  \
   --output_dir=${OUTDIR} \
   --seq_length=$SEQ_LEN \
   --n_forward=$N_FORWARD \
   --train_steps=200 \
   --eval_delay_secs=1 \
   --min_eval_frequency=60


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
2019-07-17 02:23:28.820018: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-17 02:23:28.825416: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-07-17 02:23:28.825767: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x562d9129f970 executing computations on platform Host. Devices:
2019-07-17 02:23:28.825800: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf

## Cloud ML Engine
Now to train on Cloud ML Engine with more steps and models.
Copy data to bucket first.

In [58]:
%%bash
gsutil -m rm -rf gs://${BUCKET}/energy/*
gsutil -m cp data/*.csv gs://${BUCKET}/energy

Removing gs://qwiklabs-gcp-5678415b1c0fd33e/energy/eval_8_5.csv#1563292186599406...
Removing gs://qwiklabs-gcp-5678415b1c0fd33e/energy/eval_3_1.csv#1563292186568671...
Removing gs://qwiklabs-gcp-5678415b1c0fd33e/energy/train_3_1.csv#1563292186619545...
Removing gs://qwiklabs-gcp-5678415b1c0fd33e/energy/train_8_5.csv#1563292186676943...
/ [4/4 objects] 100% Done                                                       
Operation completed over 4 objects.                                              
Copying file://data/eval_240_120.csv [Content-Type=text/csv]...
Copying file://data/eval_8_5.csv [Content-Type=text/csv]...
Copying file://data/train_240_120.csv [Content-Type=text/csv]...
Copying file://data/train_360_120.csv [Content-Type=text/csv]...
Copying file://data/train_3_1.csv [Content-Type=text/csv]...
Copying file://data/train_8_5.csv [Content-Type=text/csv]...
Copying file://data/eval_3_1.csv [Content-Type=text/csv]...
Copying file://data/eval_360_120.csv [Content-Type=text/csv]...

In [65]:
%%bash
for MODEL in linear dnn cnn rnn lstm; do
  OUTDIR=gs://${BUCKET}/project_energy/trained_${SEQ_LEN}_${N_FORWARD}/${MODEL}
  JOBNAME=energy_${MODEL}_${SEQ_LEN}_${N_FORWARD}_$(date -u +%y%m%d_%H%M%S)
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=model.task \
     --package-path=${PWD}/model \
     --job-dir=$OUTDIR \
     --scale-tier=BASIC \
     --runtime-version=$TFVERSION \
     -- \
     --model=$MODEL \
     --train_data_path="gs://${BUCKET}/energy/train_${SEQ_LEN}_${N_FORWARD}.csv" \
     --eval_data_path="gs://${BUCKET}/energy/eval_${SEQ_LEN}_${N_FORWARD}.csv"  \
     --output_dir=${OUTDIR} \
     --seq_length=$SEQ_LEN \
     --n_forward=$N_FORWARD \
     --train_steps=1000 \
     --eval_delay_secs=1 \
     --min_eval_frequency=100
done

jobId: energy_linear_240_120_190717_023956
state: QUEUED
jobId: energy_dnn_240_120_190717_023959
state: QUEUED
jobId: energy_cnn_240_120_190717_024003
state: QUEUED
jobId: energy_rnn_240_120_190717_024006
state: QUEUED
jobId: energy_lstm_240_120_190717_024009
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [energy_linear_240_120_190717_023956] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe energy_linear_240_120_190717_023956

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs energy_linear_240_120_190717_023956
CommandException: 1 files/objects could not be removed.
Job [energy_dnn_240_120_190717_023959] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe energy_dnn_240_120_190717_023959

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs energy_dnn_240_120_190717_023959
CommandException: 1 files/objects could not be removed.
Job [energy_cnn_240_120_190717_024003] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gclo